# Análise Populacional usando PySCF

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyscf
from gpu4pyscf import scf, cc
from pyscf.tools import cubegen # para exportar aquivos cube

## Molécula de H2O

In [ ]:
l0 = 0.9578 # comprimento da ligação O-H em Angstroms
angle = 104.49 * (np.pi / 180) # ângulo H

mol = pyscf.M(
    atom=f'O 0 0 0; H 0 {-l0*np.sin(angle/2)} {l0*np.cos(angle/2)}; H 0. {l0*np.sin(angle/2)} {l0*np.cos(angle/2)}',
    basis = 'aug-cc-pVTZ', unit='Angstrom', verbose=0)

Realizando um cálculo de Hartree-Fock restrito (elétrons emparelhados)

In [ ]:
mf = scf.RHF(mol)

Otimização da geometria da molécula

In [ ]:
from pyscf.geomopt.geometric_solver import optimize

gradients = []
def callback(envs):
    gradients.append(envs['gradients'])

mol_eq = optimize(mf, maxsteps=20, callback=callback)
print("Optimized coordinate:")
print(mol_eq.atom_coords())

Calculando energia pós otimização

In [ ]:
mf.kernel()

mf.energy_tot()

Energias dos orbitais e cargas de mulliken

In [ ]:
mf.analyze()

Calculando a matriz densidade

In [ ]:
dm = mf.to_cpu().make_rdm1()  # to_cpu

Exportando a densidade eletrônica

In [ ]:
# electron density
cubegen.density(mol, 'water_rho.cube', dm)

Exportando potencial eletrostático

In [ ]:
cubegen.mep(mol, 'water_esp.cube', dm)

## Análise de Bader

Deve instalar o programa de https://theory.cm.utexas.edu/henkelman/code/bader/

In [ ]:
!bader water_rho.cube

In [ ]:
Z = np.array([8,1,1])
N = np.array([9.249086,0.343737,0.343737]).round(3)  # from ACF.dat -> Bader analysis

N.sum()

Q = Z - N
print('Number of electrons from Bader analysis:', N)
print("Bader charges (e):", Q)